# Exercice Celeba

But récupérer les célébrités qui se ressemblent

In [ ]:
#Chargement des données
from os import listdir
from os.path import isfile, join
image_folder = '../input/celeba-dataset/img_align_celeba/img_align_celeba/'
onlyfiles = [image_folder + f for f in listdir(image_folder) if isfile(join(image_folder, f))]

In [ ]:
#suppression des données car le jeu de données est trop important 
del onlyfiles[5000:202599]

In [ ]:
#chargement des images à partir des chemins + préparation des images au bon format
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
list_img = []
list_label = []
for i,img_info in enumerate(onlyfiles):
    print (i, ' / ', len(onlyfiles))
    img = load_img(img_info, target_size=(224,224,3))
    img_array = img_to_array(img)
    list_img.append(img_array)
    list_label.append(img_info)
    

In [ ]:
#chargement d'un jeu réseau de neuronnes fournit par Keras et déja entrainé.
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import NASNetLarge
from keras.applications.inception_resnet_v2 import preprocess_input
model = InceptionResNetV2(pooling='avg',include_top=False, weights='imagenet', input_shape=(224,224,3))

model.summary()

In [ ]:
import numpy as np
X = np.array(list_img)
X = preprocess_input(X)

In [ ]:
#Prediction sur les données
list_finale = []
for index in range(0, len(list_img)):
    print (index, ' / ', len(list_img))
    ret_vec = model.predict(X[index].reshape(1,224,224,3))
    list_finale.append({'vec':ret_vec.reshape(-1),'original':list_img[index], 'path':list_label[index], 'index':index})

In [ ]:
#On cherche l'image défini avec la variable to_find et on l'affiche
%matplotlib inline
import matplotlib.pyplot as plt
from keras.preprocessing.image import array_to_img

to_find = 151
plt.imshow(array_to_img(list_finale[to_find]['original']))
plt.show()

In [ ]:
#on cherche les images correspondantes à l'image sélectionné
from scipy import spatial
cosine_list = []
for index_image,xt in enumerate(list_finale):
    print (index_image, ' / ', len(list_finale))
    result = 1 - spatial.distance.cosine(list_finale[to_find]['vec'].reshape(-1), xt['vec'])
    cosine_list.append(dict({'res':result, 'i':list_finale[index_image]['index']}))
from operator import itemgetter
cosine_list.sort(key=itemgetter('res'), reverse=True)

In [ ]:
#on affiche les images trouvées
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=10,figsize=(20, 4))
plt.gray()
for indice, row in enumerate(ax):
    print (cosine_list[indice]['i'])
    row.imshow(array_to_img(list_finale[cosine_list[indice]['i']]['original']))

plt.show()

In [ ]:
#on affiche l'indice de similarité
#res correspond au score de similarité 1 = identique 0 = différent
cosine_list
